In [60]:
import pandas as pd

sheets = pd.ExcelFile('data/2018bball.xlsx').sheet_names
sheets = [sheets[0]] + sheets[3:] 

In [132]:

firstClean = lambda x: x.replace('.', '').replace('Vs', 'vs').split('vs')

for sheet in sheets:


    df = pd.read_excel('data/2018bball.xlsx', header=6, sheet_name=sheet)
    df.columns = [x.replace(' ', '') for x in df.columns]
    df = df[df.TEAM.notnull()]
    df = df[['DATE', 'TEAM', 'TIME', 'LOCATION']]
    df.DATE = pd.to_datetime(df.DATE)
    df['TEAM1'] = df.TEAM.apply(lambda x: firstClean(x)[0].strip())
    df['TEAM2'] = df.TEAM.apply(lambda x: firstClean(x)[1].strip() if len(firstClean(x)) > 1 else x.strip())
    df['TIME'] = df.TIME.apply(lambda x: str(x))

    obj = {}
    for x in df.DATE.apply(lambda x: x.strftime('%Y-%m-%d')).unique():
        sliced = df[df.DATE==x]
        sliced = sliced[sliced.columns.drop('DATE')]
        i = 0
        events = []
        while i < len(sliced):
            ev = sliced.iloc[i].to_dict()
            ev["type"] = "Game"
            ev["team1score"] = "na"
            ev["team2score"] = "na"
            events += [ev]
            i += 1 
        obj[x] = {"events": events}


    
    
      
    with open('cleaned_files/' + sheet + '.json', 'w') as outfile:
        json.dump(obj, outfile)
    
    
    
    

## Working on Roster Files

In [138]:
filename = 'data/2019bballroster.xlsx'
sheets = pd.ExcelFile(filename).sheet_names


In [263]:
def getDF(sheet):
    
    print('loading up ' + sheet)
   
    i = 0
    while 'Name' not in pd.read_excel(filename, sheet_name=sheet, header = i).columns.tolist():
        print('incrementing')
        i += 1
    df = pd.read_excel(filename, header = i, sheet_name = sheet)
    print(df.columns.tolist())
    if 'Name' not in df.columns:
        df = pd.read_excel(filename, sheet_name = sheet)
    df.columns = list(map(lambda x: 'Telephone Number' if x.strip() == 'Telephone Numer' else x.strip(), df.columns.tolist()))
    return df 

In [272]:
obj = {}
for sheet in sheets:
    df = getDF(sheet)
    dicts = json.loads(df.to_json(orient='records'))
    i = 0
    while i < len(dicts):
        for attribute in ['Telephone Number', 'E-mail Address']:
            if attribute not in dicts[i]:
                dicts[i][attribute] = 'null'
            elif dicts[i][attribute] == 'None':
                dicts[i][attribute] = 'null'
        i += 1
                
    obj[sheet.strip()] = dicts

loading up All Saints-Small Fry 3rd grade
incrementing
incrementing
['Name', 'Telephone Numer ', 'Date of Birth ']
loading up All Saints-Small Fry 4th grade 
incrementing
incrementing
['Name', 'Telephone Numer ', 'Date of Birth ']
loading up Knicks - Grammar 
incrementing
incrementing
['Name', 'Telephone Numer ', 'Date of Birth ']
loading up Warriors - Grammar
['Name', 'Telephone Numer ', 'Date of Birth ']
loading up Nets - Grammar 
['Name', 'Telephone Numer ', 'Date of Birth ']
loading up Small Fry - PAL 4
['Name', 'Telephone Numer ', 'Date of Birth ']
loading up Mustard Seed Junior- Grammar 
['Name', 'Telephone Numer ', 'Date of Birth ', 'E-mail Address ']
loading up Mustard Seed  - Senior - Gramma
['Name', 'Telephone Numer ', 'Date of Birth ', 'E-mail Address ']
loading up Hud. School 6th Gr Middle Div. 
['Name', 'Telephone Numer ', 'Date of Birth ', 'E-mail Address ']
loading up Hud School 7th-8th Gr. Grammar
incrementing
['Name', 'Telephone Numer ', 'Date of Birth ', 'E-mail Addre

In [273]:
with open('cleaned_files/rosters.json', 'w') as outfile:
    json.dump(obj, outfile)

## Second Roster

In [275]:
filename = 'data/instructionalroster.xlsx'
sheets = pd.ExcelFile(filename).sheet_names


In [278]:
df = getDF(sheets[0])

loading up Instructional Basketball Roster
incrementing
incrementing
incrementing
incrementing
incrementing
incrementing
incrementing
incrementing
incrementing
['Name', 'Family ID', 'Phone', 'RPG Work Phone', 'RPG Mobile Phone', 'Email', 'Gr', 'DOB', 'Reg Date']


In [285]:
df.columns = map(lambda x: x.strip(), df.columns)

obj = json.loads(df.to_json(orient='records'))
with open('cleaned_files/instructionalRoster.json', 'w') as outfile:
    json.dump(obj, outfile)